<img src="../figs/eh_logo.png" style="width: 200px;">

# EmptyHeaded Query Language

This tutorial provides the basic information concerning our query language
The EmptyHeaded query language is standard datalog with aggregations and recursion added. We go over each of the following features of our query language.

1. [Joins](#join)
2. [Projections](#projection)
3. [Selections](#selection)
4. [Aggregations](#agg)
5. [Recursion](#recursion)

To run this tutorial you must have successfully completed the Getting Started Tutorial so that the database already exists on disk.

In [1]:
from emptyheaded import *
start()
eh = os.path.expandvars("$EMPTYHEADED_HOME")
db = Database.from_existing(eh+"/docs/notebooks/graph/db")

<a id='join'></a>
## Joins

Joins can be expressed in EmptyHeaded as in standard datalog. Simply specify the relation and the attributes that you are joining.  Emptyheaded currently supports equijoins; relations with the same attribute are joined on those matching attributes. For example consider the triangle query which performs joins over the *Edge* relation. 

In [2]:
db.eval("Triangle(a,b,c) :- Edge(a,b),Edge(b,c),Edge(a,c).")

<a id='projection'></a>
## Projections

Projections are easily expressed by simply removing the attributes that are projected from the head of the rule. For example consider the triangle query where we just find all edges that participate in a triangle. We can project away the *c* attribute. The query below does this.

In [3]:
db.eval("Triangle(a,b) :- Edge(a,b),Edge(b,c),Edge(a,c).")

<a id='selection'></a>
## Selections

EmptyHeaded currently supports equality selections by simply adding an additional condition to the body of the datalog rule. For example consider the query where we find all 4-cliques connected to a specified node. Note: equality selections currently must be projected away (i.e. not appear in the head).

In [5]:
db.eval("""FliqueSel(a,b,c,d) :- x=0,
 Edge(a,b),Edge(b,c),Edge(a,c),
 Edge(a,d),Edge(b,d),Edge(c,d),Edge(a,x).""")

<a id='agg'></a>
## Aggregations

### Overview
We add basic aggregations to EmptyHeaded following the work of [Puttagunta et al](http://arxiv.org/abs/1508.07532). The key optimization that we use in EmptyHeaded which other worst-case optimal join processing engines do not use is the ability to perform early aggregation. We use *generalized hypertree decompositions* (GHDs) as our query plans in EmptyHeaded (thus, they are our equivalent of relational algebra). This allows us to treat aggregations as tropical semirings that support standard aggregation operations like $\sum,\times,\max,\min$.  

### Annotations

These annotations support aggregations from
any semiring (a generalization of natural numbers equipped with a
notion of addition and multiplication). This enables EmptyHeaded to support
classic aggregations such as $SUM$, $MIN$, or $COUNT$, but also
enables \EH to support more complicated computations such as gradient
descent. To specify the annotation, one uses a semicolon in the head
of the rule, e.g. $q(x, y; z:int)$ specifies that each $x,y$ pair will
be associated with an integer value with alias $z$ similar to a GROUP
BY in SQL. In addition, the user expresses the aggregation operation
in the body of the rule. The user can specify an initialization value
as any expression over the tuples' values and constants, while common
aggregates have default values. A typical
count query is shown next.

### Composing the query

Now that the database is loaded lets write a simple aggregation query. We call this query the Barbell query or $B_{3,1}$ as it finds all 3-cliques connected by a path of length one. The syntax below shows that the annotation is first specified in the head (e.g. `w`) with the aggregation appearing in the body of the rule (e.g. `w:long<-[COUNT(*)]`). Let's find out how many barbells are in our Facebook dataset.


In [6]:
db.eval("""
BarbellAgg(;w) :- w:long<-[COUNT(*)],Edge(a,b),Edge(b,c),Edge(a,c),
  Edge(a,x),Edge(x,y),Edge(y,z),Edge(x,z).
""")

<a id='recursion'></a>

## Recursion

We have support for simple (1 base case, 1 recursive statement) recursive queries in the current release of EmptyHeaded and plan to expand this support in the future. In this tutorial we demonstrate the computation of PageRank using power iteration.  


In [7]:
pr="""
N(;w) :- Edge(x,y),w:long<-[SUM(x;1)].
PageRank(x;y) :- Edge(x,z),y:float<-[(1.0/N)].
PageRank(x;y)*[i=5]:-Edge(x,z),PageRank(z),InvDegree(z),
   y:float <- [0.15+0.85*SUM(z;1.0)].
"""
db.eval(pr)
pr = db.get("PageRank").getDF()
print pr

         0          1
0        0  15.227066
1        1   0.529718
2        2   0.455440
3        3   0.512322
4        4   0.571060
5        5   0.498271
6        6   0.472287
7        7   0.601821
8        8   0.610678
9        9   1.121320
10      10   0.312845
11      11   0.182558
12      12   0.182558
13      13   0.630738
14      14   0.612276
15      15   0.182558
16      16   0.324536
17      17   0.566828
18      18   0.182558
19      19   0.918776
20      20   0.631879
21      21   1.297353
22      22   0.399617
23      23   1.027091
24      24   0.502344
25      25   1.611363
26      26   1.251622
27      27   0.273024
28      28   0.552328
29      29   0.464899
...    ...        ...
4009  4009   0.609358
4010  4010   0.212187
4011  4011   0.374186
4012  4012   0.346326
4013  4013   0.466006
4014  4014   0.740338
4015  4015   0.212187
4016  4016   0.478064
4017  4017   0.665054
4018  4018   0.468058
4019  4019   0.515517
4020  4020   0.587647
4021  4021   0.671851
4022  4022

This is an example of the syntax used to express the PageRank query in EmptyHeaded. The first line specifies that we aggregate over all the edges in the graph and count the number of source nodes assuming our *Edge* relation is two-attribute relation filled with *(src,dst)* pairs. For an undirected graph this simply counts the number of nodes in the graph and assigns it to the relation *N* which is really just a scalar integer. By definition the *COUNT* aggregation and by default the*SUM* use an initialization value of *1* if the relation is not annotated. The second line of the query defines the base case for recursion. Here we simply project away the *z* attributes and assign an annotation value of *1/N* (where *N* is our scalar relation holding the number of nodes). Finally, the third line defines the recursive rule which joins the *Edge* and *InvDegree* relations inside the database with the new *PageRank* relation. We *SUM* over the *z* attribute in all of these relations. When aggregated attributes are joined with each other, their annotation values are multiplied by default using the default definition of multiplication in our semi-ring. Therefore we are performing a matrix-vector multiplication. After the aggregation, the corresponding expression for the annotation *y* is applied to each aggregated value. This is run for a fixed number (5) of iterations as specified in the head.

In [6]:
stop()